In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [3]:
#  Import and read the owid-covid-data.csv.
raw_covid_df = pd.read_csv("Resources/owid-covid-data.csv")
raw_covid_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2/24/2020,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2/25/2020,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2/26/2020,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2/27/2020,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2/28/2020,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [21]:
subset_covid = raw_covid_df[['continent', 'location', 'date', 'total_cases_per_million', 'new_cases_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'total_tests', 'total_tests_per_thousand','new_tests','new_tests_per_thousand','positive_rate', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'population','median_age', 'aged_65_older', 'aged_70_older', 'cardiovasc_death_rate', 'diabetes_prevalence', 'life_expectancy']]
subset_covid


,continent,location,date,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,total_tests_per_thousand,new_tests,...,total_vaccinations,people_vaccinated,people_fully_vaccinated,population,median_age,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,life_expectancy
0,Asia,Afghanistan,2/24/2020,0.026,0.026,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,38928341.0,18.6,2.581,1.337,597.029,9.59,64.83
1,Asia,Afghanistan,2/25/2020,0.026,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,38928341.0,18.6,2.581,1.337,597.029,9.59,64.83
2,Asia,Afghanistan,2/26/2020,0.026,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,38928341.0,18.6,2.581,1.337,597.029,9.59,64.83
3,Asia,Afghanistan,2/27/2020,0.026,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,38928341.0,18.6,2.581,1.337,597.029,9.59,64.83
4,Asia,Afghanistan,2/28/2020,0.026,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,38928341.0,18.6,2.581,1.337,597.029,9.59,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75553,Africa,Zimbabwe,3/13/2021,2453.824,3.230,100.990,0.336,NaN,NaN,NaN,...,36359.0,36359.0,NaN,14862927.0,19.6,2.822,1.882,307.846,1.82,61.49
75554,Africa,Zimbabwe,3/14/2021,2454.698,0.875,101.124,0.135,NaN,NaN,NaN,...,36359.0,36359.0,NaN,14862927.0,19.6,2.822,1.882,307.846,1.82,61.49
75555,Africa,Zimbabwe,3/15/2021,2456.044,1.346,101.191,0.067,NaN,NaN,NaN,...,37660.0,37660.0,NaN,14862927.0,19.6,2.822,1.882,307.846,1.82,61.49
75556,Africa,Zimbabwe,3/16/2021,2458.130,2.086,101.393,0.202,NaN,NaN,NaN,...,39550.0,39550.0,NaN,14862927.0,19.6,2.822,1.882,307.846,1.82,61.49


In [20]:
usa_covid = subset_covid[subset_covid['location'].str.contains('United States', na = False)]
usa_covid

,continent,location,date,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,total_tests_per_thousand,new_tests,...,total_vaccinations,people_vaccinated,people_fully_vaccinated,population,median_age,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,life_expectancy
71614,North America,United States,1/22/2020,0.003,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
71615,North America,United States,1/23/2020,0.003,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
71616,North America,United States,1/24/2020,0.006,0.003,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
71617,North America,United States,1/25/2020,0.006,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
71618,North America,United States,1/26/2020,0.015,0.009,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72030,North America,United States,3/13/2021,88822.713,160.769,1614.235,5.211,NaN,NaN,NaN,...,105703501.0,68884011.0,36929777.0,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
72031,North America,United States,3/14/2021,88938.186,115.473,1615.963,1.728,NaN,NaN,NaN,...,107060274.0,69784210.0,37459269.0,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
72032,North America,United States,3/15/2021,89109.330,171.144,1618.199,2.236,NaN,NaN,NaN,...,109081860.0,71054445.0,38335432.0,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
72033,North America,United States,3/16/2021,89271.198,161.869,1622.084,3.885,NaN,NaN,NaN,...,110737856.0,72135616.0,39042345.0,331002647.0,38.3,15.413,9.732,151.089,10.79,78.86
